# 🔄 Transfer Learning Type A & B avec modèles torchvision


Ce notebook teste plusieurs modèles pré-entraînés (`torchvision.models`) pour la classification binaire post-ouragan (Flooded/Damaged vs Undamaged), selon :

- **Type A** : Feature extractor (gelé) + nouveau classifieur
- **Type B** : Fine-tuning complet (non gelé) + nouveau classifieur

Modèles testés :
- `resnet18`
- `alexnet`
- `densenet121`


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, roc_auc_score, f1_score
import numpy as np
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_dir = "Post-hurricane"
img_size = 150
batch_size = 32
num_epochs = 5


In [ ]:

# Transforms
transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train_another"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "validation_another"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes


In [ ]:

def create_model(model_name='resnet18', type_B=False):
    base_model = getattr(models, model_name)(pretrained=True)

    # Geler les couches si Type A
    if not type_B:
        for param in base_model.parameters():
            param.requires_grad = False

    # Adapter le classifier selon l'architecture
    if model_name.startswith("resnet"):
        in_features = base_model.fc.in_features
        base_model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 1)
        )
    elif model_name.startswith("alexnet") or model_name.startswith("vgg"):
        in_features = base_model.classifier[-1].in_features
        base_model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 1)
        )
    elif model_name.startswith("densenet"):
        in_features = base_model.classifier.in_features
        base_model.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(in_features, 1)
        )
    else:
        raise ValueError("Architecture non supportée pour ce script")

    return base_model.to(device)


In [ ]:

def train_and_evaluate(model):
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.float().unsqueeze(1).to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{num_epochs} completed.")

    # Évaluation
    model.eval()
    y_true, y_prob = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = torch.sigmoid(outputs).cpu().numpy().flatten()
            y_prob.extend(probs)
            y_true.extend(labels.numpy())

    y_pred = (np.array(y_prob) > 0.5).astype(int)

    print(classification_report(y_true, y_pred, target_names=class_names))
    print(f"AUC: {roc_auc_score(y_true, y_prob):.4f}")
    print(f"F1 Score: {f1_score(y_true, y_pred):.4f}")


In [ ]:

# 🔁 Test de ResNet18 Type A (gelé)
print("🧪 ResNet18 - Type A (gelé)")
model_resnet18_A = create_model('resnet18', type_B=False)
train_and_evaluate(model_resnet18_A)

# 🔁 Test de ResNet18 Type B (fine-tuning)
print("🧪 ResNet18 - Type B (fine-tuning)")
model_resnet18_B = create_model('resnet18', type_B=True)
train_and_evaluate(model_resnet18_B)


In [ ]:

# 🔁 Test de DenseNet121 Type A
print("🧪 DenseNet121 - Type A (gelé)")
model_densenet_A = create_model('densenet121', type_B=False)
train_and_evaluate(model_densenet_A)

# 🔁 Test de DenseNet121 Type B
print("🧪 DenseNet121 - Type B (fine-tuning)")
model_densenet_B = create_model('densenet121', type_B=True)
train_and_evaluate(model_densenet_B)


In [ ]:

# 🔁 Test de AlexNet Type A
print("🧪 AlexNet - Type A (gelé)")
model_alexnet_A = create_model('alexnet', type_B=False)
train_and_evaluate(model_alexnet_A)

# 🔁 Test de AlexNet Type B
print("🧪 AlexNet - Type B (fine-tuning)")
model_alexnet_B = create_model('alexnet', type_B=True)
train_and_evaluate(model_alexnet_B)
